In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Autism_spectrum_disorder_(ASD)"
cohort = "GSE89594"

# Input paths
in_trait_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)"
in_cohort_dir = "../../input/GEO/Autism_spectrum_disorder_(ASD)/GSE89594"

# Output paths
out_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/GSE89594.csv"
out_gene_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/gene_data/GSE89594.csv"
out_clinical_data_file = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/clinical_data/GSE89594.csv"
json_path = "../../output/preprocess/Autism_spectrum_disorder_(ASD)/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability 
# Based on the background information, this dataset seems to contain gene expression data ("integrated transcriptome analysis")
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (Autism Spectrum Disorder)
trait_row = 0  # The diagnosis information is in row 0
# For age
age_row = 2  # Age information is in row 2
# For gender
gender_row = 3  # Gender information is in row 3

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait values to binary (0 for control, 1 for ASD)"""
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if "autism" in value or "asd" in value:
        return 1  # ASD is present
    elif "control" in value:
        return 0  # Control
    # Williams Syndrome is not our trait of interest
    elif "williams" in value or "ws" in value:
        return None
    else:
        return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    # Extract numeric age from strings like "age: 22y"
    if 'y' in value:
        try:
            age = int(value.replace('y', ''))
            return age
        except ValueError:
            return None
    else:
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value == "female":
        return 0
    elif value == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract and save clinical features
if trait_row is not None:
    # Create sample characteristics dictionary
    sample_characteristics_dict = {
        0: ['diagnosis: control', 'diagnosis: autism spectrum disorder (ASD)', 'diagnosis: Williams Syndrome (WS)'],
        1: ['tissue: whole blood'],
        2: ['age: 22y', 'age: 23y', 'age: 24y', 'age: 33y', 'age: 21y', 'age: 20y', 'age: 28y', 'age: 25y', 'age: 32y', 
            'age: 36y', 'age: 30y', 'age: 27y', 'age: 31y', 'age: 35y', 'age: 10y', 'age: 16y', 'age: 11y', 'age: 12y', 
            'age: 38y', 'age: 34y', 'age: 29y', 'age: 19y', 'age: 13y', 'age: 15y', 'age: 43y', 'age: 14y', 'age: 17y', 
            'age: 39y', 'age: 26y'],
        3: ['gender: female', 'gender: male']
    }
    
    # Create a proper DataFrame for geo_select_clinical_features
    # The function expects rows to represent features
    clinical_data = pd.DataFrame.from_dict(sample_characteristics_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll provide clean code for this step without any markdown or narrative text:

```python
# Review the output data and determine gene expression availability, trait, age, and gender data availability
import os
import pandas as pd
import numpy as np
import re
import json

# Load main files from this cohort
expression_files = [f for f in os.listdir(in_cohort_dir) if f.endswith('.txt') or f.endswith('.csv')]
if len(expression_files) > 0:
    main_file = os.path.join(in_cohort_dir, expression_files[0])
    # Check if file exists and try to peek at its content
    if os.path.exists(main_file):
        with open(main_file, 'r') as f:
            header = [next(f) for _ in range(10)]
        # Look for gene expression indicators
        gene_expr_indicators = ['gene', 'expression', 'RNA', 'transcript']
        is_gene_available = any(any(indicator.lower() in line.lower() for indicator in gene_expr_indicators) for line in header)
    else:
        is_gene_available = False
else:
    is_gene_available = False

# Load clinical data or characteristic information
clinical_file = os.path.join(in_cohort_dir, "clinical_data.csv")
if os.path.exists(clinical_file):
    clinical_data = pd.read_csv(clinical_file)
    print("Clinical data preview:")
    print(clinical_data.head())
    
    # Search for trait information - ASD related terms
    trait_search_terms = ['autism', 'asd', 'diagnosis', 'condition', 'disease', 'control', 'case', 'patient', 'status']
    trait_row = None
    
    # Search for age information
    age_search_terms = ['age', 'years', 'year old']
    age_row = None
    
    # Search for gender information
    gender_search_terms = ['gender', 'sex', 'male', 'female']
    gender_row = None
    
    # Check each row for trait, age, and gender information
    for i in range(len(clinical_data)):
        row_values = list(clinical_data.iloc[i])
        row_text = ' '.join([str(val).lower() for val in row_values if pd.notna(val)])
        
        # Check for trait information
        if trait_row is None and any(term in row_text for term in trait_search_terms):
            # Verify it's not constant across all samples
            values = [val for val in row_values if pd.notna(val) and val != clinical_data.columns[0]]
            unique_values = set(values)
            if len(unique_values) > 1:  # More than one unique value
                trait_row = i
        
        # Check for age information
        if age_row is None and any(term in row_text for term in age_search_terms):
            # Verify it's not constant across all samples
            values = [val for val in row_values if pd.notna(val) and val != clinical_data.columns[0]]
            unique_values = set(values)
            if len(unique_values) > 1:  # More than one unique value
                age_row = i
        
        # Check for gender information
        if gender_row is None and any(term in row_text for term in gender_search_terms):
            # Verify it's not constant across all samples
            values = [val for val in row_values if pd.notna(val) and val != clinical_data.columns[0]]
            unique_values = set(values)
            if len(unique_values) > 1:  # More than one unique value
                gender_row = i
    
    # If trait information not found, check for study design clues
    if trait_row is None:
        metadata_file = os.path.join(in_cohort_dir, "metadata.json")
        if os.path.exists(metadata_file):
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
            if 'summary' in metadata:
                summary = metadata['summary'].lower()
                if 'autism' in summary or 'asd' in summary:
                    # Look for sample groups in clinical data again with different approach
                    for i in range(len(clinical_data)):
                        row_text = ' '.join([str(val).lower() for val in clinical_data.iloc[i] if pd.notna(val)])
                        if 'group' in row_text or 'subject' in row_text or 'sample' in row_text:
                            values = [val for val in clinical_data.iloc[i] if pd.notna(val) and val != clinical_data.columns[0]]
                            unique_values = set(values)
                            if len(unique_values) > 1:
                                trait_row = i
                                break
                                
    # Define conversion functions
    def convert_trait(value):
        if pd.isna(value) or value is None:
            return None
        
        value_str = str(value).lower()
        # Extract value after colon if present
        if ':' in value_str:
            value_str = value_str.split(':', 1)[1].strip()
        
        # Convert ASD/autism/case to 1, control/normal/healthy to 0
        if any(term in value_str for term in ['asd', 'autism', 'case', 'patient', 'positive']):
            return 1
        elif any(term in value_str for term in ['control', 'normal', 'healthy', 'negative', 'non-asd']):
            return 0
        return None
    
    def convert_age(value):
        if pd.isna(value) or value is None:
            return None
        
        value_str = str(value).lower()
        # Extract value after colon if present
        if ':' in value_str:
            value_str = value_str.split(':', 1)[1].strip()
        
        # Extract numerical age using regex
        age_match = re.search(r'(\d+\.?\d*)', value_str)
        if age_match:
            try:
                return float(age_match.group(1))
            except:
                return None
        return None
    
    def convert_gender(value):
        if pd.isna(value) or value is None:
            return None
        
        value_str = str(value).lower()
        # Extract value after colon if present
        if ':' in value_str:
            value_str = value_str.split(':', 1)[1].strip()
        
        # Convert gender: female=0, male=1
        if any(term in value_str for term in ['female', 'f', 'woman', 'girl']):
            return 0
        elif any(term in value_str for term in ['male', 'm', 'man', 'boy']):
            return 1
        return None
    
    # Determine if trait data is available
    is_trait_available = trait_row is not None
    
    # Save initial validation information
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )
    
    # Extract clinical features if trait data is available
    if is_trait_available:
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data, 
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age if age_row is not None else None,
            gender_row=gender_row,
            convert_gender=convert_gender if gender_row is not None else None
        )
        
        # Preview and save clinical data
        print("Selected clinical features preview:")
        preview = preview_df(selected_clinical_df)
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.


### Step 4: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 5: Gene Identifier Review

In [ ]:
# Analyzing the gene identifiers
# The identifiers shown ('1', '2', '3'...) are numeric values, not standard gene symbols
# Human gene symbols would typically be alphanumeric like "BRCA1", "TP53", "GAPDH", etc.
# These appear to be numeric identifiers that would need to be mapped to actual gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify columns for gene identifiers and gene symbols in the annotation data
# Based on the preview, 'ID' contains identifiers that match the gene expression data
# 'GENE_SYMBOL' contains the corresponding gene symbols
probe_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'

# 2. Get the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_symbol_col)

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print a preview of the gene expression data after mapping
print("Gene expression data preview after mapping:")
print(gene_data.shape)
print(gene_data.index[:10])  # Show first 10 gene symbols


### Step 8: Data Normalization and Linking

In [ ]:
# 1. First, we need to re-establish the file paths and extract data again
# Get file paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Extract the sample characteristics from the matrix file
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Define trait, age, and gender rows based on the sample characteristics dictionary
trait_row = 0  # Diagnosis information is in row 0
age_row = 2    # Age information is in row 2 
gender_row = 3 # Gender information is in row 3

# Define conversion functions for clinical data
def convert_trait(value):
    """Convert trait values to binary (0 for control, 1 for ASD)"""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        return None
    
    if "autism" in value or "asd" in value:
        return 1  # ASD is present
    elif "control" in value:
        return 0  # Control
    # Williams Syndrome is not our trait of interest
    return None

def convert_age(value):
    """Convert age values to continuous numeric values"""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        return None
    
    # Extract numeric age from strings like "age: 22y"
    if 'y' in value:
        try:
            age = int(value.replace('y', ''))
            return age
        except ValueError:
            return None
    return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        return None
    
    if value == "female":
        return 0
    elif value == "male":
        return 1
    return None

# Extract clinical features
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Get gene expression data again
gene_data = get_genetic_data(matrix_file)

# Get gene annotation and mapping
gene_annotation = get_gene_annotation(soft_file)
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Now we continue with the normalization and linking process
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Clinical and genetic data linked: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"After handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
note = "Dataset contains ASD, control, and Williams Syndrome samples. Only ASD and control samples are used."
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=note
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked and processed data saved to {out_data_file}")
else:
    print("Dataset was determined to be unusable for trait-gene association studies.")